# Entrenamiento y uso del modelo LDA
En este notebook se cubriran los siguientes temas:
- Bigramas, trigramas y palabras comunes.
- Eliminación de palabras irrelevantes y/o poco comunes.
- Entrenamiento del modelo LDA.
- Visualización.
- Utilización del modelo LDA.

# Importando las librerías necesarias.

In [5]:
import numpy as np
import pandas as pd
import csv
import glob
import json

#Gensim

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel

#spacy Lemmatization processes

import nltk
import spacy
from nltk.corpus import stopwords

#Visualization


import pyLDAvis
import pyLDAvis.gensim_models
import pyLDAvis.gensim

Aquí empezamos a preparar los datos para el entrenamiento del modelo.

In [26]:
def load_data(file):
    csv.field_size_limit(1000000)
    with open (file, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        data = [row for row in reader]
    return data

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data,f,indent=4 )
        

In [27]:
stopwords = stopwords.words("spanish")

In [28]:
print(stopwords)

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

In [29]:
data = load_data("clean_corpus.csv")

In [30]:
words = []
for row in data:
    words.append(row["text"])

In [31]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(words)

print (data_words[0][0:20])

['buen', 'dia', 'terminar', 'reunion', 'seguridad', 'informo', 'obtener', 'informacion', 'delito', 'cometido', 'establecer', 'existir', 'sistema', 'informacion', 'reporte', 'incidencia', 'delictivo', 'diariamente', 'sistema', 'existir']


## Bigramas, trigramas y palabras comunes

In [32]:
#Bigramas y trigramas
bigrams_phrases = gensim.models.Phrases(data, min_count = 5, threshold = 50)
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data_words], threshold=50)

bigram=gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return(bigram[doc] for doc in texts)

def make_trigrams(texts):
    return(trigram[bigram[doc]] for doc in texts)

data_bigrams =  list(make_bigrams(data_words))
data_bigrams_trigrams = list(make_trigrams(data_bigrams))


In [33]:
print(data_bigrams_trigrams[0][0:20])

['buen', 'dia', 'terminar', 'reunion', 'seguridad', 'informo', 'obtener', 'informacion', 'delito_cometido', 'establecer', 'existir', 'sistema', 'informacion', 'reporte', 'incidencia_delictivo', 'diariamente', 'sistema', 'existir', 'estadistico', 'tener']


## Eliminación de palabras irrelevantes y/o poco comunes.

Utilizando la técnica TF-IDF (Term Frecuency-Inverse Document Frecuency) utilizando los bigramas y trigramas generados anteriormente, después entrenamos el modelo TF-IDF utilizando este mismo corpus. Definimos un umbral para usarlo como filtro de palabras para eliminar las que tengan una menor puntuación a esta y las que no aparecen en el propio modelo.

In [34]:
#TF-IDF Removal

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]


tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tfidf = []

for i in range(0,len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i]=new_bow


## Entrenamiento del modelo LDA.

In [37]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus, 
                                            id2word = id2word, 
                                            num_topics= 14, 
                                            random_state = 100, 
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha="auto")

## Visualización

In [38]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds = "mmds", R = 30)
vis

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.386807 -0.049005       1        1  30.029073
7      0.095730 -0.382372       2        1  14.852140
5     -0.282861 -0.293999       3        1   7.137736
4     -0.101905  0.248973       4        1   6.911386
3      0.147953 -0.115586       5        1   6.699985
2      0.050702 -0.047113       6        1   5.542796
9      0.045552  0.244384       7        1   4.975944
11    -0.069737 -0.135475       8        1   4.622699
12    -0.227757  0.116005       9        1   4.390164
13     0.221143  0.213114      10        1   3.750437
6      0.087461  0.074161      11        1   3.548297
1     -0.039873  0.057299      12        1   2.768273
0     -0.224266 -0.021101      13        1   2.492711
8     -0.088949  0.090714      14        1   2.278357, topic_info=                   Term         Freq        Total Category  logprob  loglift
13748        vacunacion  2358.000000  2358.000000  Default  30.0000  30.0000
13390             dosis  1995.000000  1995.000000  Default  29.0000  29.0000
23751           vacunar  1784.000000  1784.000000  Default  28.0000  28.0000
14668            vacuno  1609.000000  1609.000000  Default  27.0000  27.0000
22553            vacuna  1594.000000  1594.000000  Default  26.0000  26.0000
...                 ...          ...          ...      ...      ...      ...
12858  flujo_migratorio     7.558406    45.766997  Topic14  -6.2210   1.9808
8362            subasta     8.499184   118.955746  Topic14  -6.1037   1.1429
4069             tarifa     8.677208   201.173715  Topic14  -6.0830   0.6382
6864               fase     8.383954   137.739911  Topic14  -6.1174   0.9827
6813           epidemia     8.653118   380.432163  Topic14  -6.0858  -0.0017

[742 rows x 6 columns], token_table=       Topic      Freq               Term
term                                     
3226       5  0.981225             abasto
12         6  0.954265          academico
7456      13  0.988087          acarreado
2148       1  0.053271          accidente
2148       5  0.053271          accidente
...      ...       ...                ...
23310      1  0.038487  zona_arqueologico
23310      2  0.477236  zona_arqueologico
23310      3  0.456710  zona_arqueologico
23310      7  0.025658  zona_arqueologico
22572      9  0.962894              zurdo

[1206 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 8, 6, 5, 4, 3, 10, 12, 13, 14, 7, 2, 1, 9])

In [39]:
lda_model.save('my_lda_model')

## Utilización del modelo LDA.

Una vez entrenado el modelo este se carga para su utilización.

In [6]:
# Load the trained LDA model
lda_model = gensim.models.ldamodel.LdaModel.load('my_lda_model')

# Load the new data
new_data = pd.read_csv('clean_corpus.csv')

# Load the SpaCy model for tokenization
nlp = spacy.load('es_core_news_sm')

In [7]:
topics = lda_model.show_topics(num_topics=15, num_words=10)
for topic in topics:
    print(f"Topic {topic[0]}: {topic[1]}")

Topic 0: 0.033*"aeropuerto_internacional" + 0.026*"minuto" + 0.025*"aifa" + 0.018*"pasajero" + 0.018*"acarreado" + 0.015*"futbol" + 0.015*"club" + 0.012*"terminal" + 0.012*"conectividad" + 0.009*"conectar"
Topic 1: 0.027*"inteligencia" + 0.024*"cartel" + 0.022*"feminicidio" + 0.021*"estadounidense" + 0.011*"fentanilo" + 0.008*"droga" + 0.008*"bloqueado" + 0.007*"aseguramiento" + 0.006*"influyente" + 0.006*"entramado"
Topic 2: 0.011*"nuevo_leon" + 0.008*"tamaulipa" + 0.008*"productor" + 0.008*"epidemia" + 0.008*"coronavirus" + 0.006*"evento" + 0.006*"recaudacion" + 0.005*"virus" + 0.005*"transmision" + 0.005*"enfermedad"
Topic 3: 0.027*"ducto" + 0.013*"zacateco" + 0.013*"barril" + 0.013*"refineria" + 0.008*"pipar" + 0.007*"hidalgo" + 0.007*"robo_combustible" + 0.006*"culpable" + 0.005*"forense" + 0.005*"crudo"
Topic 4: 0.016*"credito" + 0.014*"salario_minimo" + 0.009*"octubre" + 0.009*"descenso" + 0.008*"ejido" + 0.007*"cofepris" + 0.007*"ruben" + 0.006*"narrativo" + 0.006*"hospitalizac

Nombramos cada uno de los topicos acorde al tema según nuestras palabras asociadas a este, creamos una bolsa de palabras e inferimos los topicos de la bolsa de palabras utilizando el modelo LDA anteriormente entrenado con todo el corpus.

In [8]:
# Create a bag of words representation of the preprocessed new corpus
new_corpus = [doc.split() for doc in new_data['text']]
new_bow_corpus = [lda_model.id2word.doc2bow(doc) for doc in new_corpus]

# Infer topics for the new corpus using the trained LDA model
new_topics = [lda_model.get_document_topics(doc, minimum_probability=0.0) for doc in new_bow_corpus]

topics_dict = {
    1: "Aeropuertos y vuelos",
    2: "Ley y crimen",
    3: "La pandemia del COVID-19 y su impacto",
    4: "La industria petrolera",
    5: "Economía",
    6: "Tren Maya",
    7: "Salud y medicina",
    8: "Reformas políticas",
    9: "Fronteras e inmigración",
    10: "Energía y minería",
    11: "Vacunación",
    12: "Problemas sociales y justicia penal",
    13: "Deportes y turismo",
    14: "Relaciones políticas entre México y Perú"
}

Obtenemos las probabilidades de los 14 topicos que nos dio en cada semana, para después guardar los resultados en un dataframe

In [9]:
# Obtener las probabilidades de los 14 tópicos para cada documento
topic_probs = []
for topics in new_topics:
    probs = [topic[1] for topic in topics]
    topic_probs.append(probs)

# Crear el dataframe de probabilidades de los 14 tópicos
df_probs = pd.DataFrame(topic_probs, columns=topics_dict.values())
df_probs.insert(0, "week", new_data["week"])

In [10]:
df_probs

week  Aeropuertos y vuelos  Ley y crimen   
0    2018-12-03              0.028691      0.015211  \
1    2018-12-10              0.525716      0.041977   
2    2018-12-24              0.025931      0.011382   
3    2018-12-31              0.017002      0.021354   
4    2019-01-07              0.030686      0.021270   
..          ...                   ...           ...   
214  2023-02-13              0.039516      0.025875   
215  2023-02-20              0.136937      0.030411   
216  2023-02-27              0.036833      0.028563   
217  2023-03-06              0.020691      0.138175   
218  2023-03-13              0.033889      0.040936   

     La pandemia del COVID-19 y su impacto  La industria petrolera  Economía   
0                                 0.015710                0.145398  0.276190  \
1                                 0.023965                0.096246  0.030492   
2                                 0.017638                0.578971  0.033275   
3                                 0.031591                0.182798  0.089951   
4                                 0.020566                0.574797  0.060565   
..                                     ...                     ...       ...   
214                               0.040953                0.058573  0.068454   
215                               0.048812                0.084357  0.055069   
216                               0.051232                0.079745  0.084962   
217                               0.048526                0.073987  0.068486   
218                               0.053135                0.105784  0.067407   

     Tren Maya  Salud y medicina  Reformas políticas  Fronteras e inmigración   
0     0.032176          0.017541            0.197006                 0.010322  \
1     0.014790          0.009365            0.036887                 0.005134   
2     0.032946          0.010113            0.071404                 0.008078   
3     0.038740          0.015976            0.078147                 0.015553   
4     0.027770          0.015156            0.061348                 0.014206   
..         ...               ...                 ...                      ...   
214   0.192499          0.023955            0.199830                 0.030543   
215   0.171948          0.019902            0.121892                 0.018386   
216   0.154437          0.037653            0.157018                 0.023832   
217   0.135085          0.028012            0.135631                 0.021693   
218   0.197475          0.040701            0.134248                 0.023316   

     Energía y minería  Vacunación  Problemas sociales y justicia penal   
0             0.026354    0.138923                             0.044397  \
1             0.053385    0.101162                             0.039852   
2             0.011284    0.076060                             0.062460   
3             0.016371    0.116781                             0.329740   
4             0.023336    0.063180                             0.042350   
..                 ...         ...                                  ...   
214           0.023150    0.153272                             0.050515   
215           0.027539    0.118835                             0.053104   
216           0.026523    0.180176                             0.041791   
217           0.059122    0.127059                             0.049951   
218           0.035138    0.126372                             0.050316   

     Deportes y turismo  Relaciones políticas entre México y Perú  
0              0.033343                                  0.018738  
1              0.020649                                  0.000382  
2              0.031380                                  0.029079  
3              0.031036                                  0.014960  
4              0.023754                                  0.021016  
..                  ...                                       ...  
214            0.062046                

Creamos un segundo dataframe donde guardemos el tópico y su lista de palabras asociada.

In [11]:
topic_words = []
for topic_num, topic_words_str in lda_model.show_topics():
    words = topic_words_str.split('+')
    word_list = []
    for word in words:
        weight, word = word.split('*')
        word = word.strip().replace('"', '')
        weight = float(weight.strip())
        word_list.append(f"{word}:{weight:.4f}")
    topic_word = {
        'palabra_asociada': topics_dict[topic_num + 1],
        'lista_palabras': word_list
    }
    topic_words.append(topic_word)

# Crear el dataframe de palabras asociadas y lista de palabras para cada tópico
df_words = pd.DataFrame(topic_words)

In [12]:
df_words

palabra_asociada   
0                   Fronteras e inmigración  \
1                          Salud y medicina   
2     La pandemia del COVID-19 y su impacto   
3                              Ley y crimen   
4                      Aeropuertos y vuelos   
5  Relaciones políticas entre México y Perú   
6       Problemas sociales y justicia penal   
7                                 Tren Maya   
8                        Reformas políticas   
9                                Vacunación   

                                      lista_palabras  
0  [centroamerico:0.0100, espanol:0.0080, cancill...  
1  [medicamento:0.0250, avion:0.0150, hospital:0....  
2  [nuevo_leon:0.0110, tamaulipa:0.0080, producto...  
3  [inteligencia:0.0270, cartel:0.0240, feminicid...  
4  [aeropuerto_internacional:0.0330, minuto:0.026...  
5  [peru:0.0600, asilo:0.0210, noviembre:0.0200, ...  
6  [penal:0.0160, medicina_tradicional:0.0160, is...  
7  [tramo:0.0350, maya:0.0340, fentanilo:0.0250, ...  
8  [bidir:0.0240, marchar:0.0180, marcha:0.0150, ...  
9  [vacunacion:0.0470, dosis:0.0400, vacunar:0.03...

Guardamos estos dos ultimos dataframes creados para su futura utilización

In [64]:
df_probs.to_csv('probs_topics.csv', index=False)
df_words.to_csv('words_topics.csv', index=False)